In [10]:
import rasterio
import numpy as np
import pandas as pd
from shapely.geometry import box

Get the "windows" needed from the TIFF file of complete USA population count dataset.
Processing complete dataset computationally expensive.

In [11]:
with rasterio.open('../original_data/usa_ppp_2012.tif') as src:
    # Retain data for only the windows that contain NYC
    windows_data = []

    # Geographical coordinate bounds of NYC
    min_lat, max_lat = 40.52763504199989, 40.91037152011096
    min_long, max_long = -74.21220034099993, -73.70134715908382

    for ji, window in src.block_windows(1):
        # Bounds of current window read from complete US TIFF file
        window_bounds = src.window_bounds(window)
        min_long_window, min_lat_window, max_long_window, max_lat_window = window_bounds

        #  Only append window data if bounds intersect with NYC bounds
        if ((max_lat + 1 >= min_lat_window >= min_lat - 1) and
            (max_lat + 1 >= max_lat_window >= min_lat - 1) and
            (max_long + 1 >= min_long_window >= min_long - 1) and
            (max_long + 1 >= max_long_window >= min_long - 1)):
            
            windows_data.append({
                'min_long': min_long_window,
                'min_lat': min_lat_window,
                'max_long': max_long_window,
                'max_lat': max_lat_window
            })

windows_bounds_df = pd.DataFrame(windows_data)
windows_bounds_df.to_csv('../interim_refined_data/usa_windows_needed.csv')

Extract all data from only the windows defined above.

In [12]:
windows_of_interest_boxes = []

# Convert data back into box object for matching to windows needed
for d in windows_data:
    min_long = d['min_long']
    min_lat = d['min_lat']
    max_long = d['max_long']
    max_lat = d['max_lat']
    
    current_box = box(min_long, min_lat, max_long, max_lat)
    windows_of_interest_boxes.append(current_box)

with rasterio.open('../original_data/usa_ppp_2012.tif') as src:
    nodata = src.nodatavals[0]  
    data = [] 

    for ji, window in src.block_windows(1):
        window_bounds = src.window_bounds(window)
        window_box = box(*window_bounds) 

        # Check if current window matches any of the windows needed
        if any(window_box.intersects(w_box) for w_box in windows_of_interest_boxes):
            window_array = src.read(window=window, indexes=1)

            if window_array.ndim == 2: 
                rows, cols = np.indices(window_array.shape, dtype=np.int32)
                rows, cols = rows.flatten(), cols.flatten()
                rows += window.row_off
                cols += window.col_off

                xs, ys = rasterio.transform.xy(src.transform, rows, cols, offset='center')
                window_flat = window_array.flatten()

                # Exclude nodata values from collected instances
                window_data = [{'latitude': y, 'longitude': x, 'pop_count': pop} for x, y, pop in zip(xs, ys, window_flat) if pop != nodata]
                data.extend(window_data)

# Dataset of all instances from windows that intersect NYC / within NYC bounds
df = pd.DataFrame(data)
df.to_csv('../interim_refined_data/pop_count_windows.csv', index=False)

Filter out the latitude & longitude coordinates outside of NYC bounds.
Some windows were only partially in bounds.
Final NYC population dataset to be merged with complete dataset. 

In [9]:
min_lat, max_lat = 40.52763504199989, 40.91037152011096
min_long, max_long = -74.21220034099993, -73.70134715908382

df = df[(df['latitude'] >= min_lat) & (df['latitude'] <= max_lat) &
        (df['longitude'] >= min_long) & (df['longitude'] <= max_long)] 

df.to_csv('../refined_data/pop_count_new_york.csv', index=False)